In [23]:
import pandas as pd
import numpy as np
import zipfile
import random

#from fonction_import_data import afficher_fichier_zip, import_data, readme

In [17]:
PROJECT_PATH = r"C:/Users/fabri/Desktop/ProjectInterPromo/"

In [18]:
data_languedoc = pd.read_csv(PROJECT_PATH + "data/DONNEES_BRUTES/DATA_FRAUDE_LANGUEDOC.csv")

In [19]:
data_languedoc

,ID,TOP_FRAUDE,TOP_FRAUDE_CARTE,TOP_FRAUDE_VIREMENT,cp_ou_pays,SECTEUR_GESTION,INDICMAIL,INDICTEL,INDICPOR,INDICSMS,...,top_produit_Services,mt_factu_ttc_M,mt_factu_ttc_M6,mt_reduc_ttc_M,mt_reduc_ttc_M6,mt_rist_ttc_M,mt_rist_ttc_M6,Fidelite_calcule,Flag_bq_principale,profil
0,835VlMiuByiRxHoFelupVn8Mg==,0,0,0,DæTT0,SECTEUR MAUGUIO MEDITERRANEE,MAIL_OPTIN,TEL_OPTIN,POR_OPTOUT,SMS_OPTIN,...,1.0,6.0,6.0,0.0,0.0,0.0,0.0,QF,1.0,Digital Ma Banque
1,835ZsRhiH9Tzd/5rhD9STRMFA==,0,0,0,lUââO,SECTEUR NARBONNE,MAIL_NEUTRE,TEL_OPTIN,POR_OPTIN,SMS_OPTIN,...,1.0,21.0,21.0,0.0,0.0,0.0,0.0,QF,1.0,actifs classiques
2,835+1SP0Ntq4Jn0Q0PuvGd5TA==,0,0,0,UwamP,SECTEUR MONTPELLIER CENTRE UNIV,MAIL_OPTIN,TEL_OPTIN,POR_OPTIN,SMS_OPTIN,...,1.0,43.0,4.0,0.0,0.0,0.0,0.0,FI,1.0,Hyper actifs
3,835mEdkZGO6o6SLjy2zPGzm/A==,0,0,0,ïQhåx,SECTEUR MONTPELLIER PIC ST LOUP,MAIL_OPTOUT,TEL_OPTIN,POR_OPTOUT,SMS_OPTIN,...,1.0,15.0,15.0,0.0,0.0,0.0,0.0,FI,1.0,Full digital
4,835txrcwy8ckP7Ye4/k38B6cA==,0,0,0,àaCTç,SECTEUR MAUGUIO MEDITERRANEE,MAIL_OPTOUT,TEL_OPTIN,POR_OPTIN,SMS_OPTOUT,...,1.0,17.0,1.0,0.0,0.0,0.0,0.0,FI,1.0,actifs classiques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584048,835mBEOByTBamgQVgQg771KDA==,1,1,0,q9åmS,SECTEUR HAUT LGDC COEUR HERAULT,MAIL_OPTIN,TEL_NEUTRE,POR_OPTIN,SMS_OPTIN,...,1.0,1.0,5.0,0.0,0.0,0.0,0.0,NF,1.0,Full digital
584049,835rWBxack6OQY9AQFqyqJd5g==,1,1,0,6lçxa,SECTEUR MAUGUIO MEDITERRANEE,MAIL_OPTIN,TEL_OPTIN,POR_OPTIN,SMS_OPTIN,...,1.0,68.0,0.0,0.0,0.0,0.0,0.0,NF,0.0,Bienvenue
584050,835F1MFghZuFjdn+sBHNuQaBw==,1,1,0,0késÎ,SECTEUR ALES CVN,MAIL_OPTIN,TEL_NEUTRE,POR_OPTIN,SMS_OPTIN,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NF,0.0,Bienvenue
584051,835HAiChYU6Al85Ed4c5I/urw==,1,1,0,f3QRg,SECTEUR HAUT LGDC COEUR HERAULT,MAIL_OPTIN,TEL_NEUTRE,POR_OPTIN,SMS_OPTIN,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,NF,0.0,Bienvenue


In [20]:
data_toulouse=pd.read_csv(PROJECT_PATH+ "data/DONNEES_BRUTES/DATA_FRAUDE_TOULOUSE.csv")

In [7]:
data_toulouse

,ID,TOP_FRAUDE,TOP_FRAUDE_CARTE,TOP_FRAUDE_VIREMENT,Date_debut_fraude,cp_ou_pays,SECTEUR_GESTION,INDICMAIL,INDICTEL,INDICPOR,...,top_produit_Services,mt_factu_ttc_M,mt_factu_ttc_M6,mt_reduc_ttc_M,mt_reduc_ttc_M6,mt_rist_ttc_M,mt_rist_ttc_M6,Fidelite_calcule,Flag_bq_principale,profil
0,8312v5IodTRgOR8/FWWjGEwsg==,0,0,0,01FEB2021,fìPÅæ,SECTEUR CENTRE,MAIL_OPTOUT,TEL_NEUTRE,POR_OPTOUT,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NF,0.0,Très peu actifs
1,831pI3Dnm1zz6NG0iCJTfcilQ==,0,0,0,01MAR2021,aLÄsb,SECTEUR EST,MAIL_OPTIN,TEL_OPTIN,POR_OPTIN,...,1.0,60.0,42.0,0.0,0.0,15.0,0.0,FI,1.0,Hyper actifs
2,831b9S86qzWBJCaqgYOVPwWMQ==,0,0,0,01MAR2021,Z0ÎuP,SECTEUR SUD,MAIL_OPTOUT,TEL_OPTIN,POR_OPTIN,...,1.0,7.0,6.0,0.0,0.0,0.0,0.0,FI,1.0,Digital Ma Banque
3,831tT9jw5y/rwwLEMZF7PyE8w==,0,0,0,01MAR2021,ëmzXM,SECTEUR NORD,MAIL_OPTOUT,TEL_NEUTRE,POR_OPTOUT,...,1.0,6.0,6.0,0.0,0.0,0.0,0.0,FI,1.0,Hyper actifs
4,8317VwmuKeVaijtrz0cTWOm4A==,0,0,0,01SEP2020,å3èJ3,SECTEUR CENTRE,MAIL_OPTIN,TEL_NEUTRE,POR_OPTIN,...,1.0,41.0,13.0,0.0,0.0,9.0,0.0,FI,1.0,Hyper actifs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265288,831zRA1wuw9mbt/KTbham8O9Q==,0,0,0,01JUL2020,fTgÂä,SECTEUR EST,MAIL_OPTOUT,TEL_NEUTRE,POR_OPTIN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,QF,0.0,Digital Web modéré
265289,831LW2dWsvzMnLio+4MuJuqZQ==,0,0,0,01NOV2020,beÃ1O,SECTEUR NORD,MAIL_NEUTRE,TEL_NEUTRE,POR_OPTIN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NF,0.0,sans contact
265290,831ZMCaimlzSFHKxaKJEDPefQ==,0,0,0,01SEP2021,orÂrË,SECTEUR NORD,MAIL_OPTOUT,TEL_OPTIN,POR_OPTOUT,...,1.0,6.0,23.0,0.0,0.0,0.0,0.0,FI,1.0,Digital Web modéré
265291,831FiYDYlJm0Y/Y+b1+3gQasw==,0,0,0,01JUL2020,3ÎEmP,SECTEUR SUD,MAIL_OPTIN,TEL_OPTIN,POR_OPTIN,...,1.0,2.0,2.0,0.0,0.0,2.0,2.0,QF,0.0,Digital Web


In [5]:
data_remplace = pd.read_csv(PROJECT_PATH + "data/Données_v0/Donnees_v0_NAN_remplaces.csv")

In [18]:
data_v0 = pd.read_csv(PROJECT_PATH + "data/Données_v0/Donnees_V0_NAN_supprimes.csv")
quali_col = []
for i, name in zip(data_v0.dtypes, data_v0.columns):
    if i == "object":
        quali_col.append(name)

print(len(data_v0.dtypes))
data_quanti = data_v0.drop(columns=quali_col + ["Unnamed: 0"])
data_quanti = data_quanti.sample(200)

257


In [19]:
np.unique(data_quanti["TOP_FRAUDE"], return_counts=True)

(array([0, 1], dtype=int64), array([199,   1], dtype=int64))

In [9]:
from sklearn.svm import OneClassSVM
X = data_quanti.drop(columns=["TOP_FRAUDE","TOP_FRAUDE_CARTE","TOP_FRAUDE_VIREMENT"])
clf = OneClassSVM(gamma='auto').fit(X)
predic_svm=clf.predict(X)
#array([-1,  1,  1,  1, -1])
#clf.score_samples(X)

In [10]:
np.unique(predic_svm, return_counts=True)

(array([-1], dtype=int64), array([200], dtype=int64))

In [11]:
from sklearn.ensemble import IsolationForest
X = data_quanti.drop(columns=["TOP_FRAUDE","TOP_FRAUDE_CARTE","TOP_FRAUDE_VIREMENT"])
clf = IsolationForest(random_state=0).fit(X)
predic_forest=clf.predict(X)

C:\Users\fabri\Anaconda3\anconda\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\Users\fabri\Anaconda3\anconda\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\Users\fabri\Anaconda3\anconda\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [12]:
np.unique(predic_forest, return_counts=True)

(array([-1,  1]), array([ 20, 180], dtype=int64))

In [14]:
from sklearn.covariance import EllipticEnvelope
X = data_quanti.drop(columns=["TOP_FRAUDE","TOP_FRAUDE_CARTE","TOP_FRAUDE_VIREMENT"])
cov = EllipticEnvelope(random_state=0).fit(X)
# predict returns 1 for an inlier and -1 for an outlier
predic_ellip=cov.predict(X)

C:\Users\fabri\Anaconda3\anconda\lib\site-packages\sklearn\covariance\robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [15]:
np.unique(predic_ellip, return_counts=True)

(array([-1,  1]), array([ 20, 180], dtype=int64))

In [21]:
from sklearn import linear_model
X = data_quanti.drop(columns=["TOP_FRAUDE","TOP_FRAUDE_CARTE","TOP_FRAUDE_VIREMENT"])
clf = linear_model.SGDOneClassSVM(random_state=42)
clf.fit(X)
predic_linear = clf.predict(X)

AttributeError: module 'sklearn.linear_model' has no attribute 'SGDOneClassSVM'

In [20]:
np.unique(predic_linear, return_counts=True)

NameError: name 'predic_linear' is not defined

In [32]:
#génération alléatoire d'un vecteur de prédiction
pred_fraud = [random.random() for i in range (20)]
fraud=np.array(pred_fraud)>0.7

array([False,  True, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])